### Configuration

In [1]:
import os
import re
import numpy as np
import pandas as pd
import datetime

from helpers_micro import wclus_to_pd

In [2]:
import config
os.chdir(config.working_directory)
os.getcwd()

'C:\\Users\\lal85\\Desktop\\Project - Sleep'

### Parameters

In [3]:
cluster_dictionary = 'Data/cluster_dictionary.csv'

# Subject 01
subj_dir = 'Cache/Sorted/Subject_01'
out_path = 'Cache/S01_spikes.csv'

# Subject 02
#subj_dir = 'Cache/Sorted/Subject_02'
#out_path = 'Cache/S02_spikes.csv'

### Munging

In [5]:
data = pd.DataFrame()

# Loop through every directory one level below the root directory
for channel_dir in os.listdir(subj_dir):

    # Find and load the Waveclus output
    wclus_output = os.path.join(subj_dir, channel_dir, 'times_' + channel_dir + '.mat')
    spikes = wclus_to_pd(wclus_output)

    # Create meta-data
    spikes[['subject', 'date', 'electrode', 'ch_num', 'laterality', 'region']] = channel_dir.split('_')
    spikes['unit_id'] = spikes['subject'] + '_' + spikes['ch_num'] + '_' + spikes['cluster_id'].astype('str')
    spikes['channel_id'] = channel_dir
    spikes.drop(['date', 'electrode'], axis = 1, inplace = True)

    # Append to the master dataframe
    data = pd.concat([data, spikes])

In [ ]:
# Load the cluster labels and merge with spikes file
cluster_labels = pd.read_table(cluster_dictionary)
spikes = spikes.merge(cluster_labels, on = ['channel_id', 'cluster_id'])

In [ ]:
data.to_csv(out_path)